In [1]:
import os 
import sys

spark_home = os.environ.get('SPARK_HOME', None)
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.4-src.zip'))
sys.path.insert(0, os.path.join(spark_home, 'python'))
execfile(os.path.join(os.environ["SPARK_HOME"], 'python/pyspark/shell.py'))

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.2.1
      /_/

Using Python version 2.7.13 (default, Dec 20 2016 23:05:08)
SparkSession available as 'spark'.


In [5]:
from pyspark.ml.clustering import KMeans

In [454]:
votes = sc.textFile('/Users/saultoscano/Desktop/desktop_march_2018/finance_course/orie6125/clustering/114_congress.csv')

In [455]:
def remove_header(itr_index, itr):
    return iter(list(itr)[1:]) if itr_index == 0 else itr
votes = votes.mapPartitionsWithIndex(remove_header).cache()


In [456]:
votes = votes.map(lambda line: [str(i + 1) + ":" + x + " " for i, x in enumerate(line.split(',')[3:])])

In [457]:
votes=votes.zipWithIndex().map(lambda line: [str(line[1]) + ' '] + line[0])

In [458]:
votes = votes.map(lambda line: (1,line + ['\n']))

In [459]:
votes = votes.map(lambda line: (line[0],''.join(line[1])))

In [460]:
votes = votes.reduceByKey(lambda v1, v2: v1+v2)

In [461]:
new_data = votes.collect()[0][1]

In [462]:
encoding = 'utf-8'

In [463]:
import io

with io.open('new_votes.txt', 'w', encoding=encoding) as file:
    file.write(new_data)

In [2]:
dataset = spark.read.format("libsvm").load("new_votes.txt")

In [3]:
dataset.count()

100

In [4]:
dataset.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(15,[1,2,3,4,7,8,...|
|  1.0|(15,[1,2,3,4,7,9,...|
|  2.0|(15,[0,3,5,7,10,1...|
|  3.0|(15,[1,2,3,4,6,7,...|
|  4.0|(15,[3,5,7,11,13]...|
|  5.0|(15,[0,3,5,7,10,1...|
|  6.0|(15,[1,2,3,4,6,7,...|
|  7.0|(15,[0,3,5,7,10,1...|
|  8.0|(15,[1,2,3,4,6,7,...|
|  9.0|(15,[0,3,5,7,10,1...|
| 10.0|(15,[0,3,5,7,10,1...|
| 11.0|(15,[1,3,4,6,7,8,...|
| 12.0|(15,[0,3,5,7,10,1...|
| 13.0|(15,[0,1,2,3,4,6,...|
| 14.0|(15,[0,3,5,7,10,1...|
| 15.0|(15,[3,5,7,11,13,...|
| 16.0|(15,[3,5,6,7,10,1...|
| 17.0|(15,[1,2,3,4,6,7,...|
| 18.0|(15,[1,2,3,4,6,7,...|
| 19.0|(15,[1,2,3,4,6,7,...|
+-----+--------------------+
only showing top 20 rows



In [9]:
# Trains a k-means model.
kmeans = KMeans().setK(2).setSeed(1)
model = kmeans.fit(dataset)

In [10]:
# Make predictions
predictions = model.transform(dataset)


In [11]:
# Shows the result.
centers = model.clusterCenters()
print("Cluster Centers: ")
for center in centers:
    print(center)

Cluster Centers: 
[ 0.01754386  0.99122807  0.93859649  0.90350877  0.95614035  0.02631579
  0.93859649  0.97368421  0.92105263  0.93859649  0.03508772  0.0877193
  0.89473684  0.0877193   0.01754386]
[ 0.73255814  0.02325581  0.          1.          0.          0.93023256
  0.02325581  1.          0.          0.02325581  0.90697674  1.          0.
  0.95348837  0.8372093 ]


## PCA

In [14]:
from pyspark.ml.feature import PCA

In [15]:
pca = PCA(k=2, inputCol="features", outputCol="pcaFeatures")

In [17]:
model = pca.fit(dataset)

In [19]:
result = model.transform(dataset).select("pcaFeatures")

In [21]:
result.show(truncate=False)

+-----------------------------------------+
|pcaFeatures                              |
+-----------------------------------------+
|[1.4008396971349262,-0.6909535632330429] |
|[0.5516619242869416,-0.15096972885667553]|
|[-1.6320015535390366,-1.2869733232725904]|
|[1.965556435159505,-1.1102863446199063]  |
|[-0.8748381563492786,0.03513165579877332]|
|[-1.6320015535390366,-1.2869733232725904]|
|[1.965556435159505,-1.1102863446199063]  |
|[-1.6320015535390366,-1.2869733232725904]|
|[1.965556435159505,-1.1102863446199063]  |
|[-1.5180660053797363,-1.0181241116454045]|
|[-1.6320015535390366,-1.2869733232725904]|
|[1.6783204377190108,-0.9536373706247238] |
|[-1.6320015535390366,-1.2869733232725904]|
|[1.8516208870002044,-1.3791355562470922] |
|[-1.6320015535390366,-1.2869733232725904]|
|[-1.1330581621844353,-0.3798362309637957]|
|[-1.1215967414694337,-0.9078569612473002]|
|[1.965556435159505,-1.1102863446199063]  |
|[1.965556435159505,-1.1102863446199063]  |
|[1.965556435159505,-1.110286344